In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS, StaticAssets
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline.data import morningstar, Fundamentals
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import SimpleMovingAverage


# Gets the 52-week high for each asset.
class High(CustomFactor):
    window_length = 252 # ~52 weeks
    inputs = [USEquityPricing.close]
    
    def compute(self, today, asset_ids, out, close_prices):
        out[:] = np.max(close_prices, axis=0)
        

def make_pipeline():
    """
    A function to create our dynamic stock selector (pipeline). Documentation
    on pipeline can be found here:
    https://www.quantopian.com/help#pipeline-title
    """
    BILLION = 1000000000

    # Base universe set to the QTradableStocksUS
    #mask = QTradableStocksUS() & Sector().notnull()
    #mask &= (Fundamentals.market_cap.latest > BILLION)
    #mask &= (
        #(Fundamentals.morningstar_sector_code.latest != 103)  # financial 
        #(Fundamentals.morningstar_sector_code.latest != 207) &  # utility
        #& (Fundamentals.morningstar_sector_code.latest != 206)  # healthcare
        #(Fundamentals.morningstar_sector_code.latest != 309) &  # energy
        #(Fundamentals.morningstar_industry_code.latest != 20533080) #&  # pharma
        #(Fundamentals.morningstar_industry_code.latest != 10217033) &  # apparel
        #(Fundamentals.morningstar_industry_group_code != 10106) &  # metals
        #(Fundamentals.morningstar_industry_group_code != 10104)  # coal
    #)

    mask = StaticAssets(symbols([
        #'hpe',
        #'bbby',
        #'atco',
        #'cnr',

        #'asna',
        #'chs',
        #'ctra',
        #'smur',
        #'fti',
        #'agro',
        #'arco',
        #'orbc',
        #'bset',
        #'nak',
        #'kirk',
        #'cine',
        'ar',
        #'true',
        #'pbpb',
        
        # cruises
        #'CUK',
        #'RCL', 
        #'NCLH',
        
        # travel services
        
        # financial
        #'GS',
        
        # auto
        #'GM',
        #'F',
        #'FCAU',
        #'GT',
        
        # tech
        #'AAPL',
        #'eric'
        #'znga'
        #'rezi'
        #'nok',
        
        
        # airlines
        #'LUV',
        #'DAL',
        #'AAL',
        #'UAL',
        #'JBLU',
        
        # energy
        #'OXY',
        #'MRO',
        #'PBF',
        #'ET',
        #'MUR',
        
        # hotels
        #'HLT',
        #'H',
        #'MAR',
        
        # retail
        #'M',
        #'ROST',
        #'GME', 
        #'KSS',
        #'DDS',
        #'clne',
        #'tues',
        #'fran',
        #'sbow',
        #'fnko',
        #'kro'
        
        #'dtea',
        #'mik',
        
        # gold
        #'EGO',
        
        
        # casinos
        #'MGM',
        #'IGT',
        
        # theme parks
        #'SIX',
        
        # materials
        #'AA',
        
        # events
        #'EB'
    ]))
    
    mask = QTradableStocksUS() & Sector().notnull()
    mask &= (Fundamentals.market_cap.latest > BILLION)
    #mask &= (Fundamentals.morningstar_industry_code.latest.element_of((31080020, 31053108)))# airlines
    #mask &= (Funda=mentals.morningstar_industry_code.latest.element_of((10217034, 10280020))) # dept stores
    #mask &= (Fundamentals.morningstar_industry_code.latest.element_of((10290050,)))  # travel services
    #mask &= (Fundamentals.morningstar_industry_code.latest.element_of((10290030,))) # lodging 
    #mask &= (Fundamentals.morningstar_industry_code.latest.element_of((10290040,)))  # casinos
    #mask &= (Fundamentals.morningstar_industry_code.latest.element_of((10200020,))) # cars
    
    #mask &= (Fundamentals.morningstar_sector_code.latest.element_of((309,))) # energy
    #mask &= (Fundamentals.morningstar_sector_code.latest.element_of((103,))) # financial
    
    
    
    #mask &= ~(Fundamentals.morningstar_sector_code.latest.element_of((104,))) # reits, RE trusts
    
    #mask &= (Fundamentals.pb_ratio.latest <= .5)
    #mask &= (Fundamentals.current_ratio.latest >= 1.0)
    #mask &= (USEquityPricing.close.latest < 10)
    mask &= (Fundamentals.cash_and_cash_equivalents.latest >= Fundamentals.market_cap.latest)
    
    mask &= (Fundamentals.quick_ratio.latest >= 1.0)
    
    ebit_per_share = Fundamentals.ebit.latest / Fundamentals.shares_outstanding.latest
    
    earnings_yield = Fundamentals.ebit.latest/Fundamentals.enterprise_value.latest
    fcf_yield = Fundamentals.cash_return.latest
    roic      = Fundamentals.roic.latest
    debt_equity = Fundamentals.total_debt_equity_ratio.latest
    div_yield = Fundamentals.trailing_dividend_yield.latest
    gross_margin = Fundamentals.gross_margin.latest
    rnd_ocf = Fundamentals.research_and_development.latest / Fundamentals.operating_cash_flow.latest
    total_yield = Fundamentals.total_yield.latest
    
    EY_rank   = earnings_yield.rank(ascending=False, mask=mask)
    FCFY_rank = fcf_yield.rank(ascending=False, mask=mask)
    roic_rank = roic          .rank(ascending=False, mask=mask)
    debt_equity_rank = debt_equity.rank(ascending=True, mask=mask)  # lower is better
    div_rank = div_yield.rank(ascending=False, mask=mask)  # higher is better
    gross_margin_rank = gross_margin.rank(ascending=False, mask=mask)
    rnd_ocf_rank = rnd_ocf.rank(ascending=False, mask=mask)
    
    # dividend and buy back yield
    total_yield_rank = total_yield.rank(ascending=False, mask=mask)
    
    
    ma50 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=50, mask=mask)
    ma200 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=200, mask=mask)
    mv_yield = ma50 / ma200
    mv_yield_rank = mv_yield.rank(ascending=False, mask=mask)
    
    sector = Fundamentals.morningstar_sector_code.latest
    industry = Fundamentals.morningstar_industry_code.latest
    
    revenue_growth = Fundamentals.revenue_growth.latest
    revenue_growth_rank = revenue_growth.rank(ascending=False, mask=mask)
        
    # Factor of yesterday's close price.
    yesterday_close = USEquityPricing.close.latest
    
    market_cap = Fundamentals.market_cap.latest
    market_cap_rank = market_cap.rank(ascending=False, mask=mask)
    ev = Fundamentals.enterprise_value.latest
    ev_rank = ev.rank(ascending=False, mask=mask)
    
    debt_assets = Fundamentals.debtto_assets.latest
    debt_assets_rank = debt_assets.rank(ascending=True, mask=mask)  # lower is better

    current_ratio = Fundamentals.current_ratio.latest
    current_ratio_rank = current_ratio.rank(ascending=False, mask=mask)  # higher is better
    
    quick_ratio = Fundamentals.quick_ratio.latest
    quick_ratio_rank = quick_ratio.rank(ascending=False, mask=mask)  # higher is better
    
    book_value_per_share = Fundamentals.book_value_per_share.latest
    tangible_book_value_per_share = Fundamentals.tangible_book_value_per_share.latest
    
    net_debt_to_ebit = Fundamentals.net_debt.latest / Fundamentals.ebit.latest
    net_debt_to_ebit_rank = net_debt_to_ebit.rank(ascending=True, mask=mask)  # lower is better
    
    cash = Fundamentals.cash_and_cash_equivalents.latest
    cash_rank = cash.rank(ascending=False, mask=mask)
    
    market_cap = Fundamentals.market_cap.latest
    
    # Our 52-week high price.
    high = High()
    
    columns = {
            'close'         : yesterday_close,
            #'sector'        : sector,
            'industry'      : industry,
        
            'book_value_per_share': book_value_per_share,
            'tangible_book_value_per_share': tangible_book_value_per_share,
            #'high52'           : high,
            'from_high52'      : yesterday_close/high,

            'earnings_yield': earnings_yield,
            'ebit_per_share'          : ebit_per_share,
            'ey_rank'       : EY_rank,
            #'roic'          : roic,
            'roic_rank'     : roic_rank,

            'debt_equity'   : debt_equity,
            'debt_equity_rank': debt_equity_rank,
            #'ev'             : ev,
            #'ev_rank'        : ev_rank,
            'current_ratio'  : current_ratio,
            #'current_ratio_rank' : current_ratio_rank,
            'quick_ratio'    : quick_ratio,
            #'quick_ratio_rank' : quick_ratio_rank,
            'debt_assets'    : debt_assets,
            #'debt_assets_rank'  : debt_assets_rank,
            'net_debt_to_ebit': net_debt_to_ebit,
            'net_debt_to_ebit_rank': net_debt_to_ebit_rank,
        
            #'revenue_growth': revenue_growth,
            #'revenue_growth_rank': revenue_growth_rank,
            'fcf_yield': fcf_yield,
            #'fcfy_rank': FCFY_rank,
            'cash': cash,
            'market_cap': market_cap,

            #'ma50'          : ma50,
            #'ma200'         : ma200,  
        }
    
    rank   = np.sum([rank for key, rank in columns.items() if key[-4:] == 'rank'])  # lower is better
    
    display_columns = {key: value for key, value in columns.items() if key[-4:] != 'rank'}
    display_columns['_rank'] = rank
    
    pipe = Pipeline(
        columns=display_columns,
        screen=mask
    )
    return pipe

today = datetime.now().date() - timedelta(days=1)

result = run_pipeline(make_pipeline(), today, today)\
            .sort_values(by='_rank', ascending=True)\
            .sort_index(axis=1)
result

# 